In [1]:
import pandas as pd

In [2]:
data_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
data_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

In [3]:
print(data_train.shape)
print(data_test.shape)

(7613, 5)
(3263, 4)


In [4]:
data_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
data_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
import re,string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [7]:
def clean_data(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"http\S+|www\S+|https\S+","",tweet)
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    tweet = re.sub(r"\d+","",tweet)
    tokens = word_tokenize(tweet)
    clena_tweets = " ".join(tokens)
    return clena_tweets

In [8]:
radom_tweet = data_train["text"][5]
radom_tweet

'#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires'

In [9]:
cleaned_tweet = clean_data(radom_tweet)
cleaned_tweet

'rockyfire update california hwy closed in both directions due to lake county fire cafire wildfires'

In [10]:
data_train["cleaned_tweet"] = data_train["text"].apply(lambda x:clean_data(x))
data_test["cleaned_tweet"] = data_test["text"].apply(lambda x:clean_data(x))

In [11]:
print(data_train.head())
print(data_test.head())

   id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target                                      cleaned_tweet  
0       1  our deeds are the reason of this earthquake ma...  
1       1              forest fire near la ronge sask canada  
2       1  all residents asked to shelter in place are be...  
3       1  people receive wildfires evacuation orders in ...  
4       1  just got sent this photo from ruby alaska as s...  
   id keyword location                                               text  \
0   0     NaN      NaN                 Just happened a terrible car crash   
1   2

In [12]:
train_tweets = data_train.cleaned_tweet.values
test_tweets = data_test.cleaned_tweet.values
train_targets = data_train.target.values

In [13]:
from keras.preprocessing.text import Tokenizer

word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(train_tweets)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [14]:
vocab_lenght = len(word_tokenizer.word_index) + 1

In [15]:
def emd(corpus):
    return word_tokenizer.texts_to_sequences(corpus)

In [16]:
from nltk.tokenize import word_tokenize
logest_in_train = max(train_tweets,key = lambda sen:len(word_tokenize(sen)))

In [17]:
logest_in_train

'if you have a son or a daughter would you like to see them going to a war with iran and come back in a body bag let the republicans know'

In [18]:
lenght_long_sen = len(word_tokenize(logest_in_train))

In [19]:
lenght_long_sen

31

In [20]:
from keras.utils import pad_sequences
paded_seq = pad_sequences(emd(train_tweets),lenght_long_sen,padding="post")
paded_for_test = pad_sequences(emd(test_tweets),lenght_long_sen,padding="post")

In [21]:
import urllib.request
url = "http://nlp.stanford.edu/data/glove.6B.zip"
loc = "/kaggle/working/glove.6B.zip"

urllib.request.urlretrieve(url,loc)

('/kaggle/working/glove.6B.zip', <http.client.HTTPMessage at 0x7bd112681810>)

In [22]:
import zipfile

zip_file = "/kaggle/working/glove.6B.zip"
extracted_dir = "/kaggle/working/"

with zipfile.ZipFile(zip_file,"r") as zip_ref:
    zip_ref.extractall(extracted_dir)

In [23]:
import os
dir = "/kaggle/working/"
files = os.listdir(dir)

for file in files:
    if file.startswith("glove.6B") and file.endswith(".txt"):
        glove_file = os.path.join(dir,file)
        

In [24]:
glove_file

'/kaggle/working/glove.6B.100d.txt'

In [25]:
import numpy as np
embeding_dict = dict()
emdeding_dim = 200
glove_file = open('/kaggle/working/glove.6B.200d.txt')
for line in glove_file:
    records = line.split()
    words = records[0]
    vector_dim = np.asarray(records[1:],dtype="float32")
    embeding_dict[words] = vector_dim
glove_file.close()    

In [26]:
embeding_matrix = np.zeros((vocab_lenght,emdeding_dim))
for word,index in word_tokenizer.word_index.items():
    embeding_matrixd = embeding_dict.get(word)
    if embeding_matrixd is not None:
        embeding_matrix[index] = embeding_matrixd
embeding_matrix.shape        

(17228, 200)

In [27]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(paded_seq,train_targets,test_size=0.2)

In [28]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout, GlobalMaxPool1D, BatchNormalization
from keras.initializers import Constant
from sklearn.model_selection import train_test_split

In [29]:
def bidiectinal_lstm():
    model = Sequential()
    model.add(Embedding(input_dim=embeding_matrix.shape[0], 
                        output_dim=embeding_matrix.shape[1], 
                        weights = [embeding_matrix], 
                        input_length=lenght_long_sen))
    model.add(Bidirectional(LSTM(lenght_long_sen, return_sequences = True, recurrent_dropout=0.2)))
    model.add(GlobalMaxPool1D())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(lenght_long_sen, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(lenght_long_sen, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [30]:
model = bidiectinal_lstm()

In [44]:
model.fit(x_train,y_train,epochs = 10,validation_data=(x_test,y_test))

Epoch 1/10
191/191 [==============================] - 4s 18ms/step - loss: 0.2044 - accuracy: 0.9328 - val_loss: 0.9816 - val_accuracy: 0.6848
Epoch 2/10
191/191 [==============================] - 4s 20ms/step - loss: 0.1961 - accuracy: 0.9389 - val_loss: 0.5643 - val_accuracy: 0.7814
Epoch 3/10
191/191 [==============================] - 4s 21ms/step - loss: 0.1782 - accuracy: 0.9440 - val_loss: 0.8338 - val_accuracy: 0.7538
Epoch 4/10
191/191 [==============================] - 4s 20ms/step - loss: 0.1708 - accuracy: 0.9479 - val_loss: 0.8808 - val_accuracy: 0.7636
Epoch 5/10
191/191 [==============================] - 4s 19ms/step - loss: 0.1525 - accuracy: 0.9496 - val_loss: 0.8149 - val_accuracy: 0.7085
Epoch 6/10
191/191 [==============================] - 4s 20ms/step - loss: 0.1501 - accuracy: 0.9511 - val_loss: 0.8480 - val_accuracy: 0.7663
Epoch 7/10
191/191 [==============================] - 4s 19ms/step - loss: 0.1429 - accuracy: 0.9548 - val_loss: 0.7256 - val_accuracy: 0.7354

In [35]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense, TimeDistributed
from tensorflow.keras.layers import Layer, Attention, Concatenate

class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        et = tf.keras.backend.squeeze(tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b), axis=-1)
        at = tf.keras.backend.softmax(et)
        at = tf.keras.backend.expand_dims(at, axis=-1)
        output = x * at
        return tf.keras.backend.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

# Input shape: (None, 31)
sentence_input = Input(shape=(31,), dtype='int32')

# Embedding layer
embedding_dim = 100
vocab_size = 10000
sentence_embedded = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=31)(sentence_input)

# Word level Bi-LSTM
word_lstm = Bidirectional(LSTM(100, return_sequences=True))(sentence_embedded)

# Attention layer at word level
word_attention = AttentionLayer()(word_lstm)

# Sentence level Bi-LSTM
sentence_lstm = Bidirectional(LSTM(100, return_sequences=True))(word_lstm)

# Attention layer at sentence level
sentence_attention = AttentionLayer()(sentence_lstm)

# Concatenate word attention and sentence attention
sentence_rep = Concatenate()([word_attention, sentence_attention])

# Output layer
output = Dense(1, activation='sigmoid')(sentence_rep)

# Define the model
model = Model(inputs=sentence_input, outputs=output)


In [36]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

Hierarchical Attention Networks 

In [37]:
model.fit(x_train,y_train,epochs = 10,validation_data=(x_test,y_test))

Epoch 1/10
191/191 [==============================] - 85s 85ms/step - loss: 0.6114 - accuracy: 0.6519 - val_loss: 0.5064 - val_accuracy: 0.7571
Epoch 2/10
191/191 [==============================] - 7s 36ms/step - loss: 0.4178 - accuracy: 0.8146 - val_loss: 0.5190 - val_accuracy: 0.8004
Epoch 3/10
191/191 [==============================] - 4s 21ms/step - loss: 0.3396 - accuracy: 0.8570 - val_loss: 0.5252 - val_accuracy: 0.7932
Epoch 4/10
191/191 [==============================] - 3s 18ms/step - loss: 0.2957 - accuracy: 0.8814 - val_loss: 0.5057 - val_accuracy: 0.7925
Epoch 5/10
191/191 [==============================] - 3s 17ms/step - loss: 0.2616 - accuracy: 0.9028 - val_loss: 0.4936 - val_accuracy: 0.7787
Epoch 6/10
191/191 [==============================] - 4s 21ms/step - loss: 0.2382 - accuracy: 0.9122 - val_loss: 0.5890 - val_accuracy: 0.7768
Epoch 7/10
191/191 [==============================] - 4s 20ms/step - loss: 0.2135 - accuracy: 0.9232 - val_loss: 0.5938 - val_accuracy: 0.772

In [40]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dropout, Dense
from tensorflow.keras.layers import LayerNormalization, MultiHeadAttention, Conv1D
from tensorflow.keras.layers import GlobalAveragePooling1D

def transformer_encoder(inputs, head_dim, num_heads, ff_dim, dropout=0):
    attention = MultiHeadAttention(num_heads=num_heads, key_dim=head_dim)(inputs, inputs)
    attention = Dropout(rate=dropout)(attention)
    attention = LayerNormalization(epsilon=1e-6)(inputs + attention)

    outputs = Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(attention)
    outputs = Conv1D(filters=inputs.shape[-1], kernel_size=1)(outputs)
    outputs = Dropout(rate=dropout)(outputs)
    outputs = LayerNormalization(epsilon=1e-6)(attention + outputs)
    return outputs

def transformer_model(input_shape, head_dim, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout=0):
    inputs = Input(shape=input_shape)
    
    # Embedding layer
    embedding_dim = 100
    vocab_size = 10000
    x = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=input_shape[0])(inputs)
    
    # Transformer Encoder blocks
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_dim, num_heads, ff_dim, dropout)
    
    # Global average pooling and MLP
    x = GlobalAveragePooling1D()(x)
    for dim in mlp_units:
        x = Dense(units=dim, activation="relu")(x)
        x = Dropout(rate=dropout)(x)
    outputs = Dense(units=1, activation="sigmoid")(x)
    
    # Create the model
    model = Model(inputs=inputs, outputs=outputs)
    return model

# Input shape: (None, 31)
input_shape = (31,)
head_dim = 64
num_heads = 4
ff_dim = 64
num_transformer_blocks = 4
mlp_units = [128, 64]
dropout = 0.1

# Create the transformer model
model = transformer_model(input_shape, head_dim, num_heads, ff_dim, num_transformer_blocks, mlp_units, dropout)


In [41]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train,y_train,epochs = 10,validation_data=(x_test,y_test))

Epoch 1/10
191/191 [==============================] - 26s 58ms/step - loss: 0.6996 - accuracy: 0.5456 - val_loss: 0.6824 - val_accuracy: 0.5732
Epoch 2/10
191/191 [==============================] - 6s 29ms/step - loss: 0.6866 - accuracy: 0.5677 - val_loss: 0.6825 - val_accuracy: 0.5732
Epoch 3/10
191/191 [==============================] - 5s 25ms/step - loss: 0.6861 - accuracy: 0.5675 - val_loss: 0.6829 - val_accuracy: 0.5732
Epoch 4/10
191/191 [==============================] - 4s 23ms/step - loss: 0.6855 - accuracy: 0.5691 - val_loss: 0.6839 - val_accuracy: 0.5732
Epoch 5/10
191/191 [==============================] - 5s 24ms/step - loss: 0.6853 - accuracy: 0.5695 - val_loss: 0.6825 - val_accuracy: 0.5732
Epoch 6/10
191/191 [==============================] - 4s 20ms/step - loss: 0.6853 - accuracy: 0.5696 - val_loss: 0.6825 - val_accuracy: 0.5732
Epoch 7/10
191/191 [==============================] - 4s 21ms/step - loss: 0.6866 - accuracy: 0.5685 - val_loss: 0.6828 - val_accuracy: 0.573

In [42]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.layers import GlobalMaxPool1D, BatchNormalization, Dropout, Activation
from tensorflow.keras.layers import Multiply, Lambda
import tensorflow.keras.backend as K

def attention_layer(inputs):
    # Calculate attention weights
    attention_weights = Dense(1, activation='tanh')(inputs)
    attention_weights = Activation('softmax')(attention_weights)
    
    # Apply attention weights to input sequences
    weighted_inputs = Multiply()([inputs, attention_weights])
    
    return weighted_inputs

def bidiectinal_lstm_with_attention(embedding_matrix, lenght_long_sen):
    model = Sequential()
    model.add(Embedding(input_dim=embedding_matrix.shape[0], 
                        output_dim=embedding_matrix.shape[1], 
                        weights=[embedding_matrix], 
                        input_length=lenght_long_sen))
    model.add(Bidirectional(LSTM(lenght_long_sen, return_sequences=True, recurrent_dropout=0.2), merge_mode='concat'))
    
    # Apply attention mechanism
    model.add(Lambda(attention_layer))
    
    model.add(GlobalMaxPool1D())
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(lenght_long_sen, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(lenght_long_sen, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [43]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train,y_train,epochs = 10,validation_data=(x_test,y_test))

Epoch 1/10
191/191 [==============================] - 22s 57ms/step - loss: 0.6851 - accuracy: 0.5680 - val_loss: 0.6839 - val_accuracy: 0.5732
Epoch 2/10
191/191 [==============================] - 6s 29ms/step - loss: 0.6848 - accuracy: 0.5695 - val_loss: 0.6824 - val_accuracy: 0.5732
Epoch 3/10
191/191 [==============================] - 5s 26ms/step - loss: 0.6848 - accuracy: 0.5698 - val_loss: 0.6840 - val_accuracy: 0.5732
Epoch 4/10
191/191 [==============================] - 5s 24ms/step - loss: 0.6840 - accuracy: 0.5701 - val_loss: 0.6785 - val_accuracy: 0.5732
Epoch 5/10
191/191 [==============================] - 5s 24ms/step - loss: 0.6289 - accuracy: 0.6499 - val_loss: 0.6394 - val_accuracy: 0.6848
Epoch 6/10
191/191 [==============================] - 4s 21ms/step - loss: 0.4678 - accuracy: 0.7984 - val_loss: 0.4674 - val_accuracy: 0.8063
Epoch 7/10
191/191 [==============================] - 4s 19ms/step - loss: 0.3806 - accuracy: 0.8499 - val_loss: 0.4750 - val_accuracy: 0.797

In [46]:
preds = model.predict(x_test)
preds = preds.round()

48/48 [==============================] - 1s 4ms/step


In [49]:
ss = pd.read_csv('/kaggle/input/nlp-getting-started/sample_submission.csv')
ss["target"] = model.predict(paded_for_test)
ss["target"] = ss['target'].apply(lambda x: 1 if x > 0.5 else 0)
ss.head()

102/102 [==============================] - 0s 4ms/step


,id,target
0,0,0
1,2,1
2,3,1
3,9,1
4,11,1


In [50]:
ss.to_csv("attn_model.csv", index=False)
